In [1]:
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
from random import randrange
# from torchsummary import summary
import os
import datetime

import sys
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
epochs = 1000
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
trainset = torch.load('../data/dataset_10steps_4ch.pt')

In [4]:
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [5]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 100, 100]
        # Output size: [batch, 3, 100, 100]
        self.encoder = nn.Sequential(
            nn.Conv2d(4, 32, 5, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, 5, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 32, 5, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(32, 4, 5, stride=1, padding=1),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4, 6, 9, stride=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 8, 7, stride=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        # print(x.size())
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.size())
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.size())
        x = torch.flatten(x, start_dim=1) # flatten all dimensions except batch
        # print(x.size())
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        # print(x.size())
        x = F.sigmoid(self.fc2(x))
        # print(x.size())
        return x

In [7]:
class StepAE(nn.Module):
    def __init__(self):
        """Variational Auto-Encoder Class"""
        super(StepAE, self).__init__()
        
        self.enc = Autoencoder()

    def step(self, x): # For actual run after training
        x = self.enc(x)
        return x

    def forward(self, x): # For training
        # Encode x to z
        recon1 = self.enc(x)
        recon2 = self.enc(recon1)
        recon3 = self.enc(recon2)
        recon4 = self.enc(recon3)
        # recon5 = self.enc(recon4)
        
        return recon1, recon2, recon3, recon4

# Training

In [8]:
disc_history_real = []
disc_history_fake = []
rme_history = []

In [9]:
# https://uos-deep-learning.tistory.com/16
def calc_gradient_penalty(netD, real_data, generated_data):
    # GP strength
    LAMBDA = 3

    b_size = real_data.size()[0]

    # Calculate interpolation
    alpha = torch.rand(b_size, 1, 1, 1)
    alpha = alpha.expand_as(real_data)
    alpha = alpha.cuda()

    interpolated = alpha * real_data.data + (1 - alpha) * generated_data.data
    interpolated = torch.autograd.Variable(interpolated, requires_grad=True)
    interpolated = interpolated.cuda()

    # Calculate probability of interpolated examples
    prob_interpolated = netD(interpolated)

    # Calculate gradients of probabilities with respect to examples
    gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                           grad_outputs=torch.ones(prob_interpolated.size()).cuda(),
                           create_graph=True, retain_graph=True)[0]

    # Gradients have shape (batch_size, num_channels, img_width, img_height),
    # so flatten to easily take norm per example in batch
    gradients = gradients.view(b_size, -1)

    # Derivatives of the gradient close to 0 can cause problems because of
    # the square root, so manually calculate norm and add epsilon
    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

    # Return gradient penalty
    return LAMBDA * ((gradients_norm - 1) ** 2).mean()

In [10]:
def train(model, discnet, train_loader, epochnow):
    model.train()
    discnet.train()
    avg_loss = 0
    for step, (x0, _, x1, _, x2, _, x3, _, x4, _) in enumerate(train_loader):
        # noisy_x = x + train_noise_level * torch.randn(*x.shape)
        # noisy_x = np.clip(noisy_x, 0., 1.)
        del _

        x0 = x0.permute(0, 3, 1, 2)
        x1 = x1.permute(0, 3, 1, 2)
        x2 = x2.permute(0, 3, 1, 2)
        x3 = x3.permute(0, 3, 1, 2)
        x4 = x4.permute(0, 3, 1, 2)

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z))
        # model.eval()
        discnet.train()
        discnet.zero_grad()
        optimizerD.zero_grad()
        r1, r2, r3, r4 = model(x0.cuda())
        
        # All real
        label_size = x1.size(0)
        real_label = 1
        label = torch.full((label_size,), real_label, dtype=torch.float, device=device)       
        for idx, x_ in enumerate([x1, x2, x3, x4]):
            output = discnet(x_.cuda()).view(-1)
            if idx == 0:
                errD_real = criterionD(output, label)
            else:
                errD_real += criterionD(output, label)
        errD_real.backward()
        out_real = output.detach().cpu().numpy()

        # All fake
        fake_label = 0
        label.fill_(fake_label)
        for idx, r_ in enumerate([r1, r2, r3, r4]):
            output = discnet(r_.detach().contiguous()).view(-1)
            if idx == 0:
                errD_fake = criterionD(output, label)
            else:
                errD_fake += criterionD(output, label)
        errD_fake.backward()

        grad_penalty = calc_gradient_penalty(netD, x1.cuda(), r1)
        grad_penalty += calc_gradient_penalty(netD, x2.cuda(), r2)
        grad_penalty += calc_gradient_penalty(netD, x3.cuda(), r3)
        grad_penalty += calc_gradient_penalty(netD, x4.cuda(), r4)
        grad_penalty = 1*grad_penalty
        grad_penalty.backward()

        optimizerD.step()
        del r1, r2, r3, r4

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        # model.train()
        discnet.eval()
        optimizer.zero_grad()
        model.zero_grad()

        r1, r2, r3, r4 = model(x0.cuda())
        label.fill_(real_label)
        for idx, r_ in enumerate([r1, r2, r3, r4]):
            output = discnet(r_.detach().contiguous()).view(-1)
            if idx == 0:
                errG = (idx+1)*criterionD(output, label)
            else:
                errG += (idx+1)*criterionD(output, label)

        loss1 = criterion(r1, x1.cuda())
        loss2 = criterion(r2, x2.cuda())
        loss3 = criterion(r3, x3.cuda())
        loss4 = criterion(r4, x4.cuda())
        # loss5 = criterion(r5, x5.cuda())
        loss = 10*(loss1 + loss2 + loss3 + loss4) + errG
        
        del x0, x1, x2, x3, x4, r1, r2, r3, r4

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item()
        loss_history.append(loss.item())

        if step%10 == 0:
            print("step{}, loss = {}, real = {}, fake = {}".format(step, loss, np.mean(out_real), (output.mean().item())), errG.mean().item())
            disc_history_real.append([out_real])
            disc_history_fake.append([output.detach().cpu().numpy()])

        if step%100 == 0:
            # sys.stdout = open(os.devnull, 'w')
            figplot(epochnow, step)
            torch.save(model_origin.state_dict(), "../data/model/" + timestampStr + "/new_stepwise_ep{}_{}.pt".format(epochnow, step))
            torch.save(netD.state_dict(), "../data/model/" + timestampStr + "/new_discnet_ep{}_{}.pt".format(epochnow, step))
            np.save("../data/model/" + timestampStr +"/disc_history_fake", disc_history_fake)
            np.save("../data/model/" + timestampStr +"/disc_history_real", disc_history_real)
            np.save("../data/model/" + timestampStr +"/loss_history", loss_history)
            # sys.stdout = sys.__stdout__

    return avg_loss / len(train_loader)

In [11]:
netD = Discriminator().to(device)
optimizerD = optim.Adam(netD.parameters(), lr=0.001)
criterionD = nn.BCEWithLogitsLoss()

In [12]:
model_origin = StepAE().to(device)
# model_origin.load_state_dict(torch.load("../data/model/27_May_2022_18_52/new_stepwise_ep5.pt"))

criterion = nn.MSELoss()
loss_history = []

In [13]:
oneloader = DataLoader(trainset, batch_size=1, shuffle=True)
torch.cuda.empty_cache()

def figplot(epochnow, step):
    model_origin.eval()
    dataiter = iter(oneloader)
    images, _, _, _, _, _, _, _, _, _ = dataiter.next()
    del _

    recon = model_origin.step(images.permute(0, 3, 1, 2).cuda())

    # del _
    torch.cuda.empty_cache()
    # get sample outputs

    steps = 100
    skip = int(steps/10)
    
    plt.figure(figsize=(6, 24))
    ax=plt.subplot(12,1,1)
    plt.imshow(images.permute(0,3,1,2).view(1,4,100,100).cpu()[0][:-1].permute(1,2,0))
    ax=plt.subplot(12,1,2)
    output = recon.detach().view(1, 4, 100, 100).cpu()[0][:-1].permute(1,2,0)
    plt.imshow(output)
    ii = 0
    for i in range(steps):
        recon = model_origin.step(recon)
        # del _
        
        if i%skip == 0:
            ii+=1
            ax = plt.subplot(12, 1, ii + 2)
            # print(i)
            # output is resized into a batch of iages
            # use detach when it's an output that requires_grad
            output = recon.detach().view(1, 4, 100, 100).cpu()[0][:-1].permute(1, 2, 0)
            # print(output.min(), output.max())
            plt.imshow(output)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            del output
            torch.cuda.empty_cache()
    plt.savefig("../data/model/"+timestampStr+"/fig_{}_{}.png".format(epochnow, step))
    torch.save(images, "../data/model/"+timestampStr+"/input_{}_{}.pt".format(epochnow,step))
    del images
    del recon
    del dataiter

In [14]:
optimizer = torch.optim.Adam(model_origin.parameters(), lr=0.0001)

import datetime
dateTimeObj = datetime.datetime.now()
timestampStr = dateTimeObj.strftime("%d_%b_%Y_%H_%M")
if timestampStr not in os.listdir("../data/model/"):
    os.mkdir("../data/model/"+timestampStr)

for epoch in range(1, epochs+1):
    epoch_loss = train(model=model_origin, discnet=netD, train_loader=trainloader, epochnow=epoch)
    print("[Epoch {}] loss:{}".format(epoch, epoch_loss))
    torch.save(model_origin.state_dict(), "../data/model/" + timestampStr + "/new_stepwise_ep{}.pt".format(epoch))
    torch.save(netD.state_dict(), "../data/model/" + timestampStr + "/new_discnet_ep{}.pt".format(epoch))

c:\Users\sbin_LAB\.conda\envs\torch\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


step0, loss = 70.63536834716797, real = 0.4985376000404358, fake = 0.5112959146499634 4.694141864776611


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


step10, loss = 61.996280670166016, real = 0.510780930519104, fake = 0.5124908089637756 4.694118022918701
step20, loss = 58.589290618896484, real = 0.5357590913772583, fake = 0.48481470346450806 4.790078163146973
step30, loss = 55.25218963623047, real = 0.5883305668830872, fake = 0.4499776363372803 4.924103736877441
step40, loss = 48.96773147583008, real = 0.5600725412368774, fake = 0.4149225950241089 5.0257744789123535
step50, loss = 41.32895278930664, real = 0.554485023021698, fake = 0.4413851499557495 4.953170299530029
step60, loss = 37.81866455078125, real = 0.4953902065753937, fake = 0.45440906286239624 4.904299736022949
step70, loss = 34.01366424560547, real = 0.5679348707199097, fake = 0.44784823060035706 4.917586326599121
step80, loss = 27.38539695739746, real = 0.4920068085193634, fake = 0.2357913702726364 5.891550064086914
step90, loss = 19.473018646240234, real = 0.36284852027893066, fake = 0.3134150803089142 5.500545024871826


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

step210, loss = 9.954021453857422, real = 0.19646170735359192, fake = 0.17710232734680176 6.079257011413574
step220, loss = 9.867599487304688, real = 0.23641636967658997, fake = 0.21306464076042175 5.9170122146606445
step230, loss = 9.32101058959961, real = 0.3078468143939972, fake = 0.2522149384021759 5.750130653381348
step240, loss = 9.418116569519043, real = 0.22476649284362793, fake = 0.22133396565914154 5.859518527984619
step250, loss = 9.13662052154541, real = 0.2146616131067276, fake = 0.24820497632026672 5.703621864318848
step260, loss = 8.875105857849121, real = 0.2672754228115082, fake = 0.3122837245464325 5.426907062530518
step270, loss = 9.098794937133789, real = 0.21766960620880127, fake = 0.21654485166072845 5.788919448852539
step280, loss = 9.470614433288574, real = 0.22431495785713196, fake = 0.19290964305400848 6.0249457359313965
step290, loss = 9.506661415100098, real = 0.21150393784046173, fake = 0.2091890424489975 5.947414398193359


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


step300, loss = 9.192069053649902, real = 0.20758971571922302, fake = 0.2071593850851059 5.9393463134765625


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


step310, loss = 9.290597915649414, real = 0.211862251162529, fake = 0.22662362456321716 5.8681488037109375
step320, loss = 9.60213851928711, real = 0.19138440489768982, fake = 0.2110813558101654 5.948289394378662
step330, loss = 9.273575782775879, real = 0.19665543735027313, fake = 0.18564194440841675 6.051437854766846
step340, loss = 9.065618515014648, real = 0.2102275937795639, fake = 0.22982794046401978 5.854373931884766


In [ ]:
plt.plot(loss_history)
plt.show()
torch.cuda.empty_cache()


In [ ]:
# torch.cuda.empty_cache()
# model_origin = StepAE().to(device)
# model_origin.load_state_dict(torch.load("cifar10.pt"))
model_origin.eval()

# Visualize

In [ ]:
model_origin = StepAE().to(device)
model_origin.load_state_dict(torch.load("../data/model/27_May_2022_18_52/new_stepwise_ep5.pt"))
model_origin.eval()